In [3]:
import xarray as xr
import matplotlib.pyplot as plt
import os
import sys
import numpy as np

proj_dir = os.path.join(os.pardir,os.pardir)
data_dir = os.path.join(proj_dir,'data')

src_dir = os.path.join(proj_dir,'src')
sys.path.append(src_dir)

from tools.log_progress import log_progress

In [2]:
def calc_antarctic_mass_loss(m,grd):
    
    s2a = 3600*24*365.25
    rhoi = 916
    
    ice_shelf = (grd.mask_rho==1) & (grd.zice<0.0) 
    vostock = (grd.lat_rho<-75) & (grd.lat_rho>-80) & (grd.lon_rho>95) & (grd.lon_rho<115)
    
    mask = ice_shelf & ~vostock
    
    dA = (1/(grd.pm*grd.pn)).where(mask)
    weights = dA/dA.sum()
    
    ismr = (m.where(mask).mean('ocean_time')*weights*s2a).sum()
    bmb = (m.where(mask).mean('ocean_time')*dA*rhoi*s2a*10**-12).sum()
    
    print('Area of all ice shelves in 10^3 km^2: ',dA.sum().values*10**-9)
    print('Area average melt rate in m/yr: ',ismr.values)
    print('Basal mass loss in Gt/a: ',bmb.values)
    
    return

In [11]:
%matplotlib notebook
grid_path = os.path.join(data_dir,'preprocessing','processed','waom10_grd.nc')
file_path = os.path.join(data_dir,'analysis','raw','waom10_M3','ocean_avg_0009.nc')
grd10 = xr.open_dataset(grid_path)
avg10 = xr.open_dataset(file_path)

calc_antarctic_mass_loss(avg10.m,grd10)

Area of all ice shelves in 10^3 km^2:  1908.8416851122743
Area average melt rate in m/yr:  1.2033297418345597
Basal mass loss in Gt/a:  2104.0208304886582


In [4]:
grid_path = os.path.join(data_dir,'preprocessing','processed','waom2_grd.nc')
file_path = os.path.join(data_dir,'analysis','raw','waom2_fix','m.nc')
grd2 = xr.open_dataset(grid_path)
avg2 = xr.open_dataset(file_path)

calc_antarctic_mass_loss(avg2.m,grd2)

OSError: [Errno 107] Transport endpoint is not connected: b'/home/ubuntu/bigStick/antarctic_melting/data/analysis/raw/waom2_fix/m.nc'

## Individual ice shelves

In [1]:
%matplotlib notebook
import pandas as pd
from matplotlib.patches import Rectangle
from scipy.spatial import KDTree

def plot_ice_shelf_areas(grd,
                         google_sheet_url='https://docs.google.com/spreadsheets/d/1BpI6iRB569kF7TxdiFT3ColNVH9zwHgNwq49Dat7Ok4/edit?usp=sharing',
                         idx_start=0,idx_end=69,labels=True):

    csv_export_url = google_sheet_url.replace('edit?', 'export?gid=0&format=csv&')
    IS = pd.read_csv(csv_export_url)

    IS[['lat_min','lat_max','lon_min','lon_max']] = IS[['lat_min','lat_max','lon_min','lon_max']].convert_objects(convert_numeric=True)
    IS['eta']=np.nan
    IS['xi']=np.nan

    if labels:  
        
        lat_flat = grd.lat_rho.stack(etaxi = ('eta_rho','xi_rho'))
        lon_flat = grd.lon_rho.stack(etaxi = ('eta_rho','xi_rho'))

        points = np.column_stack((lat_flat.values,lon_flat.values))
        tree = KDTree(points)

        def find_etaxi(lat,lon,grd):

            target = np.column_stack((lat,lon))
            dist, ind = tree.query(target)

            eta = lat_flat[ind].eta_rho.values
            xi = lat_flat[ind].xi_rho.values

            return eta,xi

        for idx,row in IS.iloc[idx_start:idx_end].iterrows():
            if row.lat_min:

                IS['eta'][idx],IS['xi'][idx] = find_etaxi((row.lat_min+row.lat_max)*0.5,(row.lon_min+row.lon_max)*0.5,grd)


    plt.close()
    fig,ax = plt.subplots(figsize=(10,7))
    grd.zice.where((grd.mask_rho==1)&(grd.zice<0)).plot(ax=ax,alpha=0.2,add_colorbar=False)

    mask_ice = (grd.mask_rho==1) & (grd.zice<-10)

    for idx,row in IS.iloc[idx_start:idx_end].iterrows(): 

        if row.lon_min<row.lon_max:
            mask_coord = (grd.lon_rho>row.lon_min) & (grd.lon_rho<=row.lon_max) & (grd.lat_rho>row.lat_min) & (grd.lat_rho<=row.lat_max)
        else:
            mask_coord = ((grd.lon_rho>row.lon_min) | (grd.lon_rho<=row.lon_max)) & (grd.lat_rho>row.lat_min) & (grd.lat_rho<=row.lat_max)

        grd.zice.where(mask_ice & mask_coord).plot(ax=ax,add_colorbar=False)
        ax.contour(mask_coord,levels=1,alpha=0.5,linewidth=0.01,colors='k')
        if labels:
            ax.text(row.xi,row.eta,row.Names,fontsize=6)

    if labels:    
        latp = grd.lat_rho.plot.contour(alpha=0.2,levels=30,add_colorbar=False,colors="k")
        plt.clabel(latp,inline=1)
        lonp = grd.lon_rho.plot.contour(alpha=0.2,levels=90,add_colorbar=False,colors="k")
        plt.clabel(lonp,inline=1)

    ax.set_aspect('equal')
    plt.tight_layout()
    plt.show()

In [2]:
plot_ice_shelf_areas(grd2)

NameError: name 'grd2' is not defined

In [7]:
def make_mask_front(grd,nb_cells):
    
    mask_rho = grd.mask_rho.values
    mask_land = np.zeros_like(mask_rho)
    mask_land[mask_rho == 0] = 1
    mask_zice = np.zeros_like(mask_land)
    mask_zice[grd.zice.values*mask_rho != 0] = 1

    mask_front = np.zeros_like(grd.mask_rho.values)

    for j in grd.eta_rho.values:
        for i in grd. xi_rho.values:
            if mask_zice[j,i] == 1:
                j_min = max(j-nb_cells,0)
                j_max = min(j+nb_cells, np.size(mask_rho,0))
                i_min = max(i-nb_cells,0)
                i_max = min(i+nb_cells+1, np.size(mask_rho,1))

                if np.any(mask_zice[j_min:j_max,i_min:i_max] + mask_land[j_min:j_max,i_min:i_max]== 0):
                        mask_front[j,i] = 1
                        
    grd['mask_front'] = (('eta_rho','xi_rho'),mask_front)
    
    return grd

In [8]:
def make_mass_loss_table(m,grd,
                         google_sheet_url='https://docs.google.com/spreadsheets/d/1BpI6iRB569kF7TxdiFT3ColNVH9zwHgNwq49Dat7Ok4/edit?usp=sharing',
                        idx_start=0,idx_end=69,include_no_front=True):
    
    csv_export_url = google_sheet_url.replace('edit?', 'export?gid=0&format=csv&')
    IS = pd.read_csv(csv_export_url)

    s2a = 3600*24*365
    rhoi = 916

    mask_ice = (grd.mask_rho==1) & (grd.zice<0)

    for idx,row in log_progress(IS.iloc[idx_start:idx_end].iterrows(),name='Ice shelf',every=1): 

        if row.lon_min<row.lon_max:
            mask_coord = (grd.lon_rho>row.lon_min) & (grd.lon_rho<=row.lon_max) & (grd.lat_rho>row.lat_min) & (grd.lat_rho<=row.lat_max)
        else:
            mask_coord = ((grd.lon_rho>row.lon_min) | (grd.lon_rho<=row.lon_max)) & (grd.lat_rho>row.lat_min) & (grd.lat_rho<=row.lat_max)

        mask_shelf = mask_ice & mask_coord
        
        dA = (1/(grd.pm*grd.pn)).where(mask_shelf)
        weights = dA/dA.sum()
        
        dA_l = dA.where(m > 0.0)
        weights_l = dA_l/dA_l.sum()
        
        dA_g = dA.where(m < 0.0)
        weights_g = dA_g/dA_g.sum()
        
        ismr2bmb = dA*rhoi*(10**-12)

        IS.at[idx,"A"] = float(dA.sum().values)*10**-9
        IS.at[idx,"ismr"] = float((m.where(mask_shelf)*weights*s2a).sum().values)
        IS.at[idx,"ismr_l"] = float((m.where(mask_shelf & (m > 0.0))*weights_l*s2a).sum().values)
        IS.at[idx,"ismr_g"] = float((m.where(mask_shelf & (m < 0.0))*weights_g*s2a).sum().values)
        IS.at[idx,"bmb"] = float((m.where(mask_shelf)*ismr2bmb*s2a).sum().values)
        IS.at[idx,"bml"] = float((m.where(mask_shelf & (m > 0.0))*ismr2bmb*s2a).sum().values)
        IS.at[idx,"bmg"] = float((m.where(mask_shelf & (m < 0.0))*ismr2bmb*s2a).sum().values)
        
        if include_no_front:
            
            dA = dA.where(grd.mask_front == 0)
            weights = dA/dA.sum()
            
            dA_l = dA.where(m > 0.0)
            weights_l = dA_l/dA_l.sum()

            dA_g = dA.where(m < 0.0)
            weights_g = dA_g/dA_g.sum()
            
            ismr2bmb = dA*rhoi*(10**-12)
            
            IS.at[idx,"A_nf"] = float(dA.sum().values)*10**-9
            IS.at[idx,"ismr_nf"] = float((m.where(mask_shelf & (grd.mask_front == 0))*weights*s2a).sum().values)
            IS.at[idx,"ismr_nf_l"] = float((m.where(mask_shelf & (grd.mask_front == 0) & (m > 0.0))*weights_l*s2a).sum().values)
            IS.at[idx,"ismr_nf_g"] = float((m.where(mask_shelf & (grd.mask_front == 0) & (m < 0.0))*weights_g*s2a).sum().values)
            IS.at[idx,"bmb_nf"] = float((m.where(mask_shelf & (grd.mask_front == 0))*ismr2bmb*s2a).sum().values)
            IS.at[idx,"bml_nf"] = float((m.where(mask_shelf & (grd.mask_front == 0) & (m > 0.0))*ismr2bmb*s2a).sum().values)
            IS.at[idx,"bmg_nf"] = float((m.where(mask_shelf & (grd.mask_front == 0) & (m < 0.0))*ismr2bmb*s2a).sum().values)

        
    return IS

In [9]:
def merge_patches(df,name1,name2,name):
    p1 = df.loc[df['Names'] == name1]
    p2 = df.loc[df['Names'] == name2]
    
    m = {'Names':name,'A':np.nan,'bmb':np.nan,'bml':np.nan,'bmg':np.nan,'ismr':np.nan,'ismr_l':np.nan,'ismr_g':np.nan,
         'A_nf':np.nan,'bmb_nf':np.nan,'bml_nf':np.nan,'bmg_nf':np.nan,'ismr_nf':np.nan,'ismr_nf_l':np.nan,'ismr_nf_g':np.nan,}
    for key in ['A','bmb','bml','bmg','A_nf','bmb_nf','bml_nf','bmg_nf']:
        m[key] = p1[key].values + p2[key].values
    for key in ['ismr','ismr_l','ismr_g','ismr_nf','ismr_nf_l','ismr_nf_g']:    
        m[key] = ((p1[key]*p1.A_nf).values + (p2[key]*p2.A_nf).values) / m['A_nf']
    
    df = pd.concat([df,pd.DataFrame(m)],ignore_index=True)

    return df

In [10]:
grd2=make_mask_front(grd2,2)

In [18]:
IS = make_mass_loss_table(avg2.m.mean('ocean_time'),grd2,include_no_front=True)

In [19]:
IS

,Names,lat_min,lon_min,lat_max,lon_max,A,ismr,ismr_l,ismr_g,bmb,bml,bmg,A_nf,ismr_nf,ismr_nf_l,ismr_nf_g,bmb_nf,bml_nf,bmg_nf
0,Larsen G,-74.70,-62.55,-74.36,-61.35,0.440667,0.118401,0.197868,-0.214649,0.047793,0.064484,-0.016691,0.375970,0.152739,0.197535,-0.122997,0.052602,0.058522,-0.005920
1,Larsen F,-74.44,-61.96,-73.85,-60.55,0.960125,0.273902,0.447932,-0.199301,0.240890,0.288020,-0.047130,0.730129,0.336099,0.537056,-0.236963,0.224783,0.265929,-0.041146
2,Larsen E,-73.72,-62.61,-73.11,-60.05,1.334646,0.746632,1.149081,-0.442076,0.912784,1.049480,-0.136696,1.117473,0.780331,1.281518,-0.460565,0.798751,0.934381,-0.135630
3,Larsen D,-73.00,-62.70,-69.40,-59.30,14.426196,0.227548,0.483377,-0.297641,3.006915,4.295250,-1.288335,12.952375,0.220719,0.502983,-0.302526,2.618692,3.876437,-1.257745
4,Larsen C,-69.30,-65.50,-66.10,-60.00,55.656520,0.257308,0.315833,-0.219646,13.117930,14.341789,-1.223859,54.066916,0.223933,0.280166,-0.219646,11.090351,12.314210,-1.223859
5,Larsen B,-66.10,-62.75,-65.00,-59.00,4.450565,0.797374,0.812282,-0.053070,3.250669,3.254396,-0.003727,3.185164,0.505737,0.519520,-0.053070,1.475543,1.479270,-0.003727
6,Wordie,-69.49,-68.24,-68.75,-66.86,0.961659,0.457917,0.461762,-0.008484,0.403369,0.403430,-0.000061,0.533900,0.263961,0.268033,-0.008484,0.129091,0.129152,-0.000061
7,Wilkins,-71.50,-75.00,-69.70,-69.50,15.115858,1.820281,1.834274,-0.141305,25.203838,25.217697,-0.013859,12.995174,1.403745,1.416580,-0.141305,16.709586,16.723445,-0.013859
8,Bach,-72.38,-75.37,-71.50,-70.30,5.327156,3.718738,4.397840,-0.087196,18.146229,18.210654,-0.064425,4.569202,2.894508,3.526250,-0.070170,12.114642,12.166231,-0.051589
9,George VI 1,-73.80,-74.50,-72.60,-67.00,12.912626,7.583949,7.600818,-1.447199,89.702690,89.734602,-0.031913,12.146634,7.535868,7.550643,-1.395290,83.846457,83.872096,-0.025639


In [24]:
IS_merged = merge_patches(IS,'George VI 1','George VI 2','George VI')
IS_merged = merge_patches(IS_merged,'Ronne','Filchner','Ronne-Filchner')
IS_merged = merge_patches(IS_merged,'Moscow University','Totten','Moscow Uni + Totten')
IS_merged = merge_patches(IS_merged,'Brunt/Stancomb','Riiser-Larsen','Brunt + Riiser-Larsen')
IS_merged = merge_patches(IS_merged,'Fimbul','Jelbart','Fimbul + Jelbart')
IS_merged = merge_patches(IS_merged,'Ross West','Ross East','Ross')

/home/ubuntu/bigStick/anaconda3/envs/antarctic_melting/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [25]:
IS_out = IS_merged[['Names','A','bmb','ismr',
                    'A_nf','bmb_nf','ismr_nf',
                    'bml','bmg','ismr_l','ismr_g',
                    'bml_nf','bmg_nf','ismr_nf_l','ismr_nf_g',
                    'lon_min','lat_min','lon_max','lat_max']] 

pd.options.display.float_format = '{:,.2f}'.format

IS_out

,Names,A,bmb,ismr,A_nf,bmb_nf,ismr_nf,bml,bmg,ismr_l,ismr_g,bml_nf,bmg_nf,ismr_nf_l,ismr_nf_g,lon_min,lat_min,lon_max,lat_max
0,Larsen G,0.44,0.05,0.12,0.38,0.05,0.15,0.06,-0.02,0.20,-0.21,0.06,-0.01,0.20,-0.12,-62.55,-74.70,-61.35,-74.36
1,Larsen F,0.96,0.24,0.27,0.73,0.22,0.34,0.29,-0.05,0.45,-0.20,0.27,-0.04,0.54,-0.24,-61.96,-74.44,-60.55,-73.85
2,Larsen E,1.33,0.91,0.75,1.12,0.80,0.78,1.05,-0.14,1.15,-0.44,0.93,-0.14,1.28,-0.46,-62.61,-73.72,-60.05,-73.11
3,Larsen D,14.43,3.01,0.23,12.95,2.62,0.22,4.30,-1.29,0.48,-0.30,3.88,-1.26,0.50,-0.30,-62.70,-73.00,-59.30,-69.40
4,Larsen C,55.66,13.12,0.26,54.07,11.09,0.22,14.34,-1.22,0.32,-0.22,12.31,-1.22,0.28,-0.22,-65.50,-69.30,-60.00,-66.10
5,Larsen B,4.45,3.25,0.80,3.19,1.48,0.51,3.25,-0.00,0.81,-0.05,1.48,-0.00,0.52,-0.05,-62.75,-66.10,-59.00,-65.00
6,Wordie,0.96,0.40,0.46,0.53,0.13,0.26,0.40,-0.00,0.46,-0.01,0.13,-0.00,0.27,-0.01,-68.24,-69.49,-66.86,-68.75
7,Wilkins,15.12,25.20,1.82,13.00,16.71,1.40,25.22,-0.01,1.83,-0.14,16.72,-0.01,1.42,-0.14,-75.00,-71.50,-69.50,-69.70
8,Bach,5.33,18.15,3.72,4.57,12.11,2.89,18.21,-0.06,4.40,-0.09,12.17,-0.05,3.53,-0.07,-75.37,-72.38,-70.30,-71.50
9,George VI 1,12.91,89.70,7.58,12.15,83.85,7.54,89.73,-0.03,7.60,-1.45,83.87,-0.03,7.55,-1.40,-74.50,-73.80,-67.00,-72.60


In [26]:
out_path = os.path.join(proj_dir,'reports','tables','2km_mass_loss.csv')
IS_out.to_csv(out_path)